Steps required for uploading Dataset from myDrive to the colab notebook.

In [1]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
link1 = 'https://drive.google.com/open?id=1ysUmb4S4ObVeoPxoUaK59s99aYU_0L7j'
fluff, id_train = link1.split('=')
print (id_train) # Verify that you have everything after '='

1ysUmb4S4ObVeoPxoUaK59s99aYU_0L7j


In [3]:
downloaded = drive.CreateFile({'id':id_train}) 
downloaded.GetContentFile('train.tsv')

In [4]:
link2 = 'https://drive.google.com/open?id=1E2UhC_sIAK7DdG9_aE7C54sZ8L-47hoE'
fluff, id_test = link2.split('=')
print (id_test) # Verify that you have everything after '='

1E2UhC_sIAK7DdG9_aE7C54sZ8L-47hoE


In [5]:
downloaded = drive.CreateFile({'id':id_test}) 
downloaded.GetContentFile('test.tsv')


In [6]:
link3 = 'https://drive.google.com/open?id=1oFDFKIcDgHNFsDP2-pUmyq2gqVkaxGI6'
fluff, id_sub = link3.split('=')
print (id_sub) # Verify that you have everything after '='

1oFDFKIcDgHNFsDP2-pUmyq2gqVkaxGI6


In [7]:
downloaded = drive.CreateFile({'id':id_sub}) 
downloaded.GetContentFile('sub.csv')

**Importing the required libraries and functions**

In [8]:
import numpy as np 
import pandas as pd 
import nltk
import os
import gc
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
#pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_colwidth', -1)

**Loading the train and test files**

In [9]:
train = pd.read_csv('train.tsv', encoding='utf8', error_bad_lines=False, sep='\t')
test = pd.read_csv('test.tsv', encoding='utf8', error_bad_lines=False, sep='\t')

A sneak peek at the training and testing dataset

In [10]:
train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [11]:
train.shape

(156060, 4)

In [12]:
test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine effort .
1,156062,8545,An intermittently pleasing but mostly routine effort
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [13]:
test.shape

(66292, 3)

Adding Sentiment column to test datset and joining train and test for preprocessing



In [14]:
test['Sentiment']=-999
test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine effort .,-999
1,156062,8545,An intermittently pleasing but mostly routine effort,-999
2,156063,8545,An,-999
3,156064,8545,intermittently pleasing but mostly routine effort,-999
4,156065,8545,intermittently pleasing but mostly routine,-999


In [15]:
df=pd.concat([train,test],ignore_index=True)
print(df.shape)
df.tail()

(222352, 4)


,PhraseId,SentenceId,Phrase,Sentiment
222347,222348,11855,"A long-winded , predictable scenario .",-999
222348,222349,11855,"A long-winded , predictable scenario",-999
222349,222350,11855,"A long-winded ,",-999
222350,222351,11855,A long-winded,-999
222351,222352,11855,predictable scenario,-999


common preprocessing step is to check for nulls, and fill the null values with something before proceeding to the next steps. If you leave the null values intact, it will trip you up at the modelling stage later

In [16]:
df.isnull().any()

PhraseId      False
SentenceId    False
Phrase        False
Sentiment     False
dtype: bool

cleaning review

In [17]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re

In [18]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        #review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [19]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [20]:
# Run next two lines if they are not installed after that run df code
# nltk.download('wordnet')
# nltk.download('punkt')


df['clean_review']=clean_review(df.Phrase.values)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment,clean_review
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


**seperating train and test dataset**



In [21]:
df_train=df[df.Sentiment!=-999]
df_train.shape

(156060, 5)

In [22]:
df_test=df[df.Sentiment==-999]
df_test.drop('Sentiment',axis=1,inplace=True)
print(df_test.shape)

(66292, 4)


In [23]:
df_test.head()

,PhraseId,SentenceId,Phrase,clean_review
156060,156061,8545,An intermittently pleasing but mostly routine effort .,an intermittently pleasing but mostly routine effort
156061,156062,8545,An intermittently pleasing but mostly routine effort,an intermittently pleasing but mostly routine effort
156062,156063,8545,An,an
156063,156064,8545,intermittently pleasing but mostly routine effort,intermittently pleasing but mostly routine effort
156064,156065,8545,intermittently pleasing but mostly routine,intermittently pleasing but mostly routine


Splitting Train dataset into train and 20% validation set



In [24]:
train_text=df_train.clean_review.values
test_text=df_test.clean_review.values
target=df_train.Sentiment.values
y=to_categorical(target)
print(train_text.shape,target.shape,y.shape)

(156060,) (156060,) (156060, 5)


In [25]:
X_train_text,X_val_text,y_train,y_val=train_test_split(train_text,y,test_size=0.2,stratify=y,random_state=123)
print(X_train_text.shape,y_train.shape)
print(X_val_text.shape,y_val.shape)

(124848,) (124848, 5)
(31212,) (31212, 5)


Finding number of unique words in the train set

In [26]:
all_words=' '.join(X_train_text)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

13732

Finding maximum length of a review in the train set

In [27]:
r_len=[]
for text in X_train_text:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

48

**Building Keras LSTM model**

In [28]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 3
num_classes=5

Tokenize text

In [29]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_text))
X_train = tokenizer.texts_to_sequences(X_train_text)
X_val = tokenizer.texts_to_sequences(X_val_text)
X_test = tokenizer.texts_to_sequences(test_text)

Sequence padding

In [30]:
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_val.shape,X_test.shape)

(124848, 48) (31212, 48) (66292, 48)


**1. LSTM Model**

In [31]:
model1=Sequential()
model1.add(Embedding(max_features,100,mask_zero=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         1373200   
_________________________________________________________________
lstm (LSTM)                  (None, None, 64)          42240     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 1,428,021
Trainable params: 1,428,021
Non-trainable params: 0
_________________________________________________________________


In [32]:
%%time
history1=model1.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, 
                    batch_size=batch_size, verbose=1)

Epoch 1/3
976/976 [==============================] - 259s 265ms/step - loss: 1.0973 - accuracy: 0.5702 - val_loss: 0.8841 - val_accuracy: 0.6427
Epoch 2/3
976/976 [==============================] - 255s 261ms/step - loss: 0.8307 - accuracy: 0.6622 - val_loss: 0.8282 - val_accuracy: 0.6623
Epoch 3/3
976/976 [==============================] - 254s 260ms/step - loss: 0.7649 - accuracy: 0.6864 - val_loss: 0.8183 - val_accuracy: 0.6643
CPU times: user 22min 41s, sys: 1min 27s, total: 24min 9s
Wall time: 12min 54s


In [33]:
y_pred1=model1.predict_classes(X_test,verbose=1)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
2072/2072 [==============================] - 34s 17ms/step


In [34]:
sub=pd.read_csv('sub.csv')
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [35]:
sub.Sentiment=y_pred1
sub.to_csv('sub1.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


**2. CNN**

In [36]:
model2= Sequential()
model2.add(Embedding(max_features,100,input_length=max_words))
model2.add(Dropout(0.2))

model2.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model2.add(GlobalMaxPooling1D())

model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(num_classes,activation='softmax'))


model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
dropout (Dropout)            (None, 48, 100)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 48, 64)            19264     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                

In [37]:
%%time
history2=model2.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs,
                    batch_size=batch_size, verbose=1)

Epoch 1/3
976/976 [==============================] - 56s 57ms/step - loss: 0.9959 - accuracy: 0.5991 - val_loss: 0.8531 - val_accuracy: 0.6519
Epoch 2/3
976/976 [==============================] - 55s 56ms/step - loss: 0.7831 - accuracy: 0.6783 - val_loss: 0.8033 - val_accuracy: 0.6688
Epoch 3/3
976/976 [==============================] - 55s 56ms/step - loss: 0.7002 - accuracy: 0.7078 - val_loss: 0.8015 - val_accuracy: 0.6677
CPU times: user 4min 31s, sys: 15.1 s, total: 4min 46s
Wall time: 2min 46s


In [38]:
y_pred2=model2.predict_classes(X_test, verbose=1)

2072/2072 [==============================] - 6s 3ms/step


In [39]:
sub.Sentiment=y_pred2
sub.to_csv('sub2.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,3
1,156062,3
2,156063,2
3,156064,2
4,156065,2


**3. CNN + GRU**

In [40]:
model3= Sequential()
model3.add(Embedding(max_features,100,input_length=max_words))
model3.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.25))
model3.add(GRU(128,return_sequences=True))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(128,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(5,activation='softmax'))
model3.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 48, 64)            19264     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 24, 64)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 64)            0         
_________________________________________________________________
gru (GRU)                    (None, 24, 128)           74496     
_________________________________________________________________
dropout_3 (Dropout)          (None, 24, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 3072)             

In [41]:
%%time
history3=model3.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, 
                    batch_size=batch_size, verbose=1)

Epoch 1/3
976/976 [==============================] - 121s 124ms/step - loss: 1.0195 - accuracy: 0.5929 - val_loss: 0.8584 - val_accuracy: 0.6415
Epoch 2/3
976/976 [==============================] - 122s 125ms/step - loss: 0.8106 - accuracy: 0.6675 - val_loss: 0.8127 - val_accuracy: 0.6655
Epoch 3/3
976/976 [==============================] - 123s 126ms/step - loss: 0.7353 - accuracy: 0.6925 - val_loss: 0.8084 - val_accuracy: 0.6739
CPU times: user 10min 26s, sys: 32.7 s, total: 10min 58s
Wall time: 6min 8s


In [42]:
y_pred3=model3.predict_classes(X_test, verbose=1)

2072/2072 [==============================] - 20s 9ms/step


In [43]:
sub.Sentiment=y_pred3
sub.to_csv('sub3.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


**4. Bidirectional GRU**

In [44]:
model4 = Sequential()

model4.add(Embedding(max_features, 100, input_length=max_words))
model4.add(SpatialDropout1D(0.25))
model4.add(Bidirectional(GRU(128)))
model4.add(Dropout(0.5))

model4.add(Dense(5, activation='softmax'))
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 48, 100)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               176640    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 1285      
Total params: 1,551,125
Trainable params: 1,551,125
Non-trainable params: 0
_________________________________________________________________


In [45]:
%%time
history4=model4.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, 
                    batch_size=batch_size, verbose=1)

Epoch 1/3
976/976 [==============================] - 295s 302ms/step - loss: 1.0063 - accuracy: 0.5950 - val_loss: 0.8517 - val_accuracy: 0.6511
Epoch 2/3
976/976 [==============================] - 296s 304ms/step - loss: 0.8053 - accuracy: 0.6700 - val_loss: 0.8137 - val_accuracy: 0.6694
Epoch 3/3
976/976 [==============================] - 298s 306ms/step - loss: 0.7385 - accuracy: 0.6951 - val_loss: 0.8126 - val_accuracy: 0.6722
CPU times: user 25min 52s, sys: 1min 54s, total: 27min 47s
Wall time: 14min 54s


In [46]:
y_pred4=model4.predict_classes(X_test, verbose=1)

2072/2072 [==============================] - 40s 19ms/step


In [47]:
sub.Sentiment=y_pred4
sub.to_csv('sub4.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


**5. Glove word embedding**

In [48]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-10-12 09:29:24--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-10-12 09:29:25--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-10-12 09:29:25--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [49]:
import zipfile
zip_ref = zipfile.ZipFile('glove.6B.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [50]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
    
def get_embed_mat(EMBEDDING_FILE, max_features,embed_dim):
    # word vectors
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))
    print('Found %s word vectors.' % len(embeddings_index))

    # embedding matrix
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    all_embs = np.stack(embeddings_index.values()) #for random init
    embedding_matrix = np.random.normal(all_embs.mean(), all_embs.std(), 
                                        (num_words, embed_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    max_features = embedding_matrix.shape[0]
    
    return embedding_matrix

In [51]:
# embedding matrix
EMBEDDING_FILE = 'glove.6B.100d.txt'
embed_dim = 100 #word vector dim
embedding_matrix = get_embed_mat(EMBEDDING_FILE,max_features,embed_dim)
print(embedding_matrix.shape)


Found 400000 word vectors.
(13732, 100)


In [52]:
model5 = Sequential()
model5.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model5.add(SpatialDropout1D(0.25))
model5.add(Bidirectional(GRU(128,return_sequences=True)))
model5.add(Bidirectional(GRU(64,return_sequences=False)))
model5.add(Dropout(0.5))
model5.add(Dense(num_classes, activation='softmax'))
model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 48, 100)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 48, 256)           176640    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               123648    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 645       
Total params: 1,674,133
Trainable params: 1,674,133
Non-trainable params: 0
____________________________________________

In [53]:
%%time
history5=model5.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=4, 
                    batch_size=batch_size, verbose=1)

Epoch 1/4
976/976 [==============================] - 503s 515ms/step - loss: 0.9942 - accuracy: 0.5900 - val_loss: 0.8578 - val_accuracy: 0.6447
Epoch 2/4
976/976 [==============================] - 501s 514ms/step - loss: 0.8482 - accuracy: 0.6487 - val_loss: 0.7926 - val_accuracy: 0.6748
Epoch 3/4
976/976 [==============================] - 505s 517ms/step - loss: 0.7884 - accuracy: 0.6732 - val_loss: 0.7784 - val_accuracy: 0.6794
Epoch 4/4
976/976 [==============================] - 505s 517ms/step - loss: 0.7470 - accuracy: 0.6901 - val_loss: 0.7639 - val_accuracy: 0.6839
CPU times: user 58min 50s, sys: 4min 37s, total: 1h 3min 28s
Wall time: 33min 41s


In [54]:
y_pred5=model5.predict_classes(X_test, verbose=1)

2072/2072 [==============================] - 72s 35ms/step


In [55]:
sub.Sentiment=y_pred5
sub.to_csv('sub5.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [56]:
sub_all=pd.DataFrame({'model1':y_pred1,'model2':y_pred2,'model3':y_pred3,'model4':y_pred4,'model5':y_pred5})
pred_mode=sub_all.agg('mode',axis=1)[0].values
sub_all.head()

,model1,model2,model3,model4,model5
0,2,3,2,2,2
1,2,3,2,2,2
2,2,2,2,2,2
3,2,2,2,2,2
4,2,2,2,2,2


In [57]:
pred_mode=[int(i) for i in pred_mode]

In [58]:
sub.Sentiment=pred_mode
sub.to_csv('sub_mode.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
